In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import models
from torchvision import transforms
from tqdm import tqdm
import pandas as pd
import numpy as np
import cv2
import time
from PIL import Image
import glob
from torchsummary import summary
from collections import OrderedDict

In [ ]:
l = glob.glob('/scratch/groups/rubin/rvignav/input/train/*.jpeg') # '/scratch/groups/rubin/stylized-input/*.jpeg'
for i in range(4):
    a = cv2.imread(l[i])
    print(l[i])
    
    a = cv2.resize(a, (256,256))
    plt.imshow(a)
    plt.show()
    scale = 300
    a = cv2.addWeighted(a, 4, cv2.GaussianBlur(a, (0,0), scale/30), -4, 128)
    b = np.zeros(a.shape)
    cv2.circle(b, (int(a.shape[1]/2), int(a.shape[0]/2)), int(scale*0.9), (1,1,1), -1, 8, 0)
    a = a*b + 128*(1-b)
    
    plt.imshow(a)
    plt.show()

In [ ]:
def check_cuda():
    is_cuda = False
    if torch.cuda.is_available():
        is_cuda = True
    return is_cuda

def scaleRadius(img, scale):
    return cv2.resize(img, (256,256))

class DRDataset(Dataset):
    def __init__(self, csv_path, transform=None):
        # Transforms
        self.to_tensor = transforms.ToTensor()
        self.transform = transform
        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Calculate len
        self.data_len = len(self.image_arr)

    def __getitem__(self, index):     
        single_image_name = '/scratch/groups/rubin/rvignav/input/train/' + self.image_arr[index] + '.jpeg'

        try:
            a = cv2.imread(single_image_name)
            a = cv2.resize(a, (256,256))
        except:
            single_image_name = '/scratch/groups/rubin/rvignav/input/train/2604_left.jpeg'
            a = cv2.imread(single_image_name)
            a = cv2.resize(a, (256,256))
        
        scale = 300
        a = scaleRadius(a, scale)
        a = cv2.addWeighted(a, 4, cv2.GaussianBlur(a, (0,0), scale/30), -4, 128)
        b = np.zeros(a.shape)
        cv2.circle(b, (int(a.shape[1]/2), int(a.shape[0]/2)), int(scale*0.9), (1,1,1), -1, 8, 0)
        a = a*b + 128*(1-b)
        a = cv2.resize(a, (256,256))
        
        img_as_tensor = self.to_tensor(a)
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len
    
full_dataset = DRDataset('/home/users/rvignav/trainLabels_balanced.csv') #, transform=train_transform

train_size = int(0.9 * len(full_dataset)) 
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print("Train: " + str(len(train_dataset)))
print("Val: " + str(len(val_dataset)))

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                          shuffle=True, num_workers=4, pin_memory=True)

valid_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                          shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
"""
0 - No DR
1 - Mild
2 - Moderate
3 - Severe
4 - Proliferative DR
"""
classes = ['none', 'mild', 'moderate', 'severe', 'proliferative']

for data, target in train_data_loader:
    for i in range(0, 2):  
        im = data[i]
        im = torch.squeeze(im)
        plt.imshow(np.transpose(im.numpy(), (1, 2, 0)), cmap='gray')
        plt.show()
        print(str(target[i]) + ": " + classes[(int)(target[i])])
    break

In [ ]:
model = models.resnet50(pretrained=True, progress=False)
model.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(2048, 512)),
                           ('relu', nn.ReLU()), 
                           ('dropout', nn.Dropout(p=0.5)), #0.337
                           ('fc2', nn.Linear(512, 5)),
                           ('output', nn.LogSoftmax(dim=1))
                             ]))

is_cuda = check_cuda()
if is_cuda:
    model.cuda()
    
print(summary(model, (3, 256, 256)))

In [ ]:
def validate(model, valid_data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valid_data_loader:
            images, labels = data
            images = np.array(images, int) 
            images = torch.Tensor(images).float()
            labels = np.array(labels, int) 
            labels = torch.Tensor(labels).long()
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 5

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.1)
for epoch in tqdm(range(num_epochs)): 
    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        inputs, labels = data
        inputs = np.array(inputs, int) 
        inputs = torch.Tensor(inputs).float()
        labels = np.array(labels, int) 
        labels = torch.Tensor(labels).long()
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
    
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    
    torch.save(model, '/scratch/groups/rubin/adam_balanced_notstylized_' + str(epoch+1) + 'epochs')
    
    print("train_acc: " + str(validate(model, train_data_loader)))
    print("val_acc: " + str(validate(model, valid_data_loader)))
    
print('Finished Training')

In [ ]:
torch.save(model, '/scratch/groups/rubin/final_adam_balanced_notstylized')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_data_loader:
        images, labels = data
        images = np.array(images, int) 
        images = torch.Tensor(images).float()
        labels = np.array(labels, int) 
        labels = torch.Tensor(labels).long()
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
!pip install -U --quiet scikit-learn
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
model = torch.load('/scratch/groups/rubin/adam_balanced_notstylized_2epochs')

y_true = []
y_pred = []
y_tot = []

classes = ['none', 'mild', 'moderate', 'severe', 'proliferative']
with torch.no_grad():
    for data in valid_data_loader:
        images, labels = data
        images = np.array(images, int) 
        images = torch.Tensor(images).float()
        labels = np.array(labels, int) 
        labels = torch.Tensor(labels).long()
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        sm = torch.nn.Softmax()
        probs = sm(outputs)
        for i in range(len(labels)):
            y_true.append(int(labels[i].item()))
            y_pred.append(int(predicted[i].item()))
            y_tot.append(probs[i].cpu().numpy())

np.savetxt('y_true_final_adam_balanced_notstylized.csv', np.asarray(y_true), delimiter=',')
np.savetxt('y_tot_final_adam_balanced_notstylized.csv', np.asarray(y_tot), delimiter=',')

print(classification_report(y_true, y_pred, target_names=classes))